# Exploratory Data Analysis

Understanding the dataset to explore how the data present in the database and if there is a need of creating some aggrigated tables that can help with :

* Vendor Selection for profitability
* Product Pricing Optimization

In [4]:
import pandas as pd
import sqlite3

In [5]:
# Creating DataBase Connection

conn = sqlite3.connect('inventory.db')

In [6]:
# Checking tables present in the database

tables = pd.read_sql_query("select name from sqlite_master where type='table'",conn)
tables

,name


In [7]:
pd.read_sql("select count(*) from sales",conn)

DatabaseError: Execution failed on sql 'select count(*) from sales': no such table: sales

In [ ]:
for table in tables['name']:
    print('-'*50,f'{table}','-'*50)
    print('Count of Records :',pd.read_sql(f"select count(*) as count from {table}",conn)['count'].values[0])
    display(pd.read_sql(f"select * from {table} limit 5",conn))

In [ ]:
purchases = pd.read_sql_query("select * from purchases where VendorNumber = 4466",conn)
purchases

In [ ]:
purchase_prices = pd.read_sql_query("select * from purchase_prices where VendorNumber = 4466",conn)
purchase_prices

In [ ]:
vendor_invoice = pd.read_sql_query("select * from vendor_invoice where VendorNumber = 4466",conn)
vendor_invoice

In [ ]:
sales = pd.read_sql_query("select * from sales where VendorNo = 4466",conn)
sales

In [ ]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

In [ ]:
vendor_invoice['PONumber'].nunique()

In [ ]:
purchase_prices

In [ ]:
vendor_invoice.columns

In [ ]:
sales

In [ ]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()

- The purchases table contain actual purchase Data, including the date of purchase, products(brands) purchased by vendors, the amount paid (in dollars),
  and the Quantity purchased.
- The purchase price column is derived from the purchase_prices table, which provides product-wise actual and purchase prices. The combination of vendor
  and brand is unique inkhis table.
- The vendor_invoice table aggregates data from the purchases table, summarizing quantity and dollar amounts, along with an additional column for          freight.This table maintains uniqueness based on vendor and PO number.
- The sales table captures actual sales transactions, detailing the brands purchased by vendors, the quantity sold, the selling price, and the revenue     earned.

As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:
- purchase transactions made by vendors
- sales transaction data
- freight costs for each vendor
- actual product prices from vendors

In [ ]:
vendor_invoice.columns

In [ ]:
freight_summery = pd.read_sql_query(""" select VendorNumber, sum(Freight) as FreightCost from vendor_invoice group by VendorNumber""",conn)

In [ ]:
freight_summery

In [ ]:
purchases.columns

In [ ]:
purchase_prices.columns

In [ ]:
pd.read_sql_query(""" select
p.VendorNumber,
p.VendorName,
p.Brand,
p.PurchasePrice,
pp.Volume,
pp.Price as ActualPrice,
sum(p.Quantity) as TotalPurchaseQuantity,
sum(p.Dollars) as TotalPurchaseDollers
from purchases p
join purchase_prices pp
on p.Brand = pp.Brand
where p.PurchasePrice > 0
group by p.VendorNumber,p.VendorName,p.Brand
order by TotalPurchaseDollers
""",conn)

In [ ]:
sales.columns

In [ ]:
pd.read_sql_query("""select
VendorNo,
Brand,
SUM(SalesDollars) as TotalSalesDollars,
SUM(SalesPrice) as TotalSalesPrice,
SUM(SalesQuantity) as TotatSalesQuanity,
SUM(ExciseTax) as TotalExciseTax
FROM sales
GROUP BY VendorNo, Brand
ORDER BY TotalSalesDollars
""",conn)

In [ ]:
Vendor_sales_summery = pd.read_sql_query("""WITH FreightSumary AS (
    SELECT
        VendorNumber,
        Sum(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),    

PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
       on p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),


SalesSummary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)

SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSumary fs  
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC""" ,conn)

In [ ]:
Vendor_sales_summery

This query generates a vendor-wise sales and purchase summery, which is valuable for :

### Performance Optimization :

- the query involves heavy joins and aggregations on large datasets like sales and purchases.
- Storing the pre-aggregated results avoids repeated expensive computations.
- Helps in analyzing sales, Purchase and pricing for different vendor and brands.
- Future Benefits of Storing this data for faster Dashboarding & Reporting.
- Instead of running expensive queries each time, dashboards can fetch data quickly from vendor_sales_summary.

In [ ]:
Vendor_sales_summery.dtypes

In [ ]:
Vendor_sales_summery.isnull().sum()

In [ ]:
Vendor_sales_summery['VendorName'].unique()

In [ ]:
Vendor_sales_summery['Description'].unique()

In [ ]:
Vendor_sales_summery['Volume'] = Vendor_sales_summery['Volume'].astype('float64')

In [ ]:
Vendor_sales_summery.fillna(0,inplace = True)

In [ ]:
Vendor_sales_summery['VendorName'] = Vendor_sales_summery['VendorName'].str.strip()

In [ ]:
Vendor_sales_summery.dtypes

In [ ]:
Vendor_sales_summery['VendorName'].unique()

In [ ]:
Vendor_sales_summery.isnull().sum()

In [ ]:
Vendor_sales_summery['GrossProfit'] = Vendor_sales_summery['TotalSalesDollars'] - Vendor_sales_summery['TotalPurchaseDollars']

In [ ]:
Vendor_sales_summery

In [ ]:
Vendor_sales_summery['ProfitMargin'] = (Vendor_sales_summery['GrossProfit'] / Vendor_sales_summery['TotalSalesDollars']) * 100

In [ ]:
Vendor_sales_summery['StockTurnover'] = Vendor_sales_summery['TotalSalesQuantity']/Vendor_sales_summery['TotalPurchaseQuantity']

In [ ]:
Vendor_sales_summery['SalestoPurchaseRatio'] = Vendor_sales_summery['TotalSalesDollars']/Vendor_sales_summery['TotalPurchaseDollars']

In [ ]:
cursor = conn.cursor()

In [ ]:
Vendor_sales_summery.columns 

In [ ]:
cursor.execute("""CREATE TABLE Vendor_sales_summery(
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL(10,2),
    Volume,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL (15,2),
    FreightCost DECIMAL (15,2),
    GrossProfit DECIMAL (15,2),
    ProfitMargin DECIMAL (15,2),
    StockTurnover DECIMAL (15,2),
    SalestoPurchaseRatio DECIMAL (15,2),
    PRIMARY KEY (VendorNumber, Brand)
    );
    """)

In [ ]:
pd.read_sql_query("select * from Vendor_sales_summery",conn) 

In [ ]:
Vendor_sales_summery.to_sql('Vendor_sales_summery',conn,if_exists = 'replace', index = False)

In [ ]:
pd.read_sql_query("select * from Vendor_sales_summery",conn) 

In [ ]:
import sqlite3
import pandas as pd
from ingestion_db import ingest_db
import logging
import time

logging.basicConfig(
    filename="logs/get_vendor_summary.log",
    level=logging.DEBUG,
    format= "%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
) 

start = time.time()

def create_vendor_summery(conn):
    ''' This function will merge the different tables to get the overall vendor summery and adding new columns in the resultant data'''
    Vendor_sales_summery = pd.read_sql_query("""WITH FreightSumary AS (
    SELECT
        VendorNumber,
        Sum(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),    

    PurchaseSummary AS (
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price AS ActualPrice,
            pp.Volume,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases p
        JOIN purchase_prices pp
           on p.Brand = pp.Brand
        WHERE p.PurchasePrice > 0
        GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
    ),
    
    
    SalesSummary AS (
        SELECT
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    
    SELECT
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.PurchasePrice,
        ps.ActualPrice,
        ps.Volume,
        ps.TotalPurchaseQuantity,
        ps.TotalPurchaseDollars,
        ss.TotalSalesQuantity,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalExciseTax,
        fs.FreightCost
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss
        ON ps.VendorNumber = ss.VendorNo
        AND ps.Brand = ss.Brand
    LEFT JOIN FreightSumary fs  
        ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC""" ,conn)

    return Vendor_sales_summery



def clean_data(df):
    '''This function will clean Data'''
    # changing datatype to float 
    df['Volume'] = df ['Volume'].astype('float')

    # filling missing value to 0
    df.fillna(0, inplace = True)

    # removing spaces from categorial columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # Creating new columns for better Analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['ProfitMargin'] = (df['GrossProfit'] / df['TotalSalesDollars']) * 100
    df['StockTurnover'] = df['TotalSalesQuantity']/df['TotalPurchaseQuantity']
    df['SalestoPurchaseRatio'] = df['TotalSalesDollars']/df['TotalPurchaseDollars']

    return df


if __name__ == '__main__':
    # creating database connection
    conn = sqlite3.connect('inventory.db')

    logging.info('Creating Vendor Sumary Table....')
    summery_df = create_vendor_summery(conn)
    logging.info(summery_df.head())

    logging. info('Cleaning Data....')
    clean_df = clean_data(summery_df)
    logging.info(clean_df.head())
    
    logging.info( 'Ingesting data....')
    ingest_db(clean_df,'Vendor_sales_summery',conn)
    logging. info('Completed')

end = time.time()